In [2]:
import matplotlib.pyplot as plt

from ALLCools.clustering import tsne
from ALLCools.plot import *
from ALLCools.integration import SeuratIntegration
from wmb import brain, cemba,aibs

import scanpy as sc
import anndata
from harmonypy import run_harmony

## Parameters

In [3]:
resolution = 0.5

## Load

In [1]:
adata_merge = anndata.read_h5ad('final.h5ad')
adata_merge

## Harmony for embedding

In [ ]:
# this step can be skipped

In [5]:
# # 30 mins to run 4M + 3K cell

# ho = run_harmony(data_mat=adata_merge.obsm['X_pca_integrate'],
#                  meta_data=adata_merge.obs,
#                  nclust=50,
#                  vars_use=['Modality'], 
#                  max_iter_harmony=30)

In [6]:
# adata_merge.obsm['X_harmony'] = ho.Z_corr.T

### TSNE

In [7]:
# # 1 hour to run 4M + 3K cell
# tsne(adata_merge, obsm='X_pca_integrate')

### UMAP

In [8]:
# 15 min to run 4M + 3K cell
sc.pp.neighbors(adata_merge, use_rep='X_pca_integrate')

In [9]:
# 2 hours to run 4M + 3K cell
sc.tl.leiden(adata_merge, resolution=resolution)

In [10]:
# 2 hours to run 4M + 3K cell
# 4 hours to run 4M + 3K cell if using spectral init, the init step is very slow
min_dist = max(0.1, 1 - adata_merge.shape[0] / 60000)
try:
    sc.tl.paga(adata_merge, groups='leiden')
    sc.pl.paga(adata_merge, plot=False)
    sc.tl.umap(adata_merge, min_dist=min_dist, init_pos='paga')
except Exception:
    print('Init with PAGA failed, use default spectral init')
    sc.tl.umap(adata_merge, min_dist=min_dist)

## Plot

In [11]:
mc_data = adata_merge[adata_merge.obs['Modality'] == 'mC']
merfish_data = adata_merge[adata_merge.obs['Modality'] == 'merfish']

## Save

In [13]:
adata_merge.write_h5ad('final_with_coords.h5ad')

In [14]:
adata_merge

AnnData object with n_obs × n_vars = 60198 × 448
    obs: 'blank_count', 'n_counts', 'n_genes', 'Modality', 'L4Region', 'CellType', 'DissectionRegion', 'leiden'
    var: 'chrom-mC', 'cov_mean-mC', 'end-mC', 'start-mC', 'cef-mC', 'id-merfish'
    uns: 'neighbors', 'leiden', 'paga', 'leiden_sizes', 'umap'
    obsm: 'X_pca_integrate', 'X_umap'
    obsp: 'distances', 'connectivities'

In [15]:
import subprocess
subprocess.run(['rm', '-f', 'final.h5ad'])

CompletedProcess(args=['rm', '-f', 'final.h5ad'], returncode=0)